In [ ]:
import re, nltk

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from wordcloud import WordCloud
from PIL import Image

In [ ]:
with open("resources/Quo_vadis.txt", encoding="utf8") as myfile:
    data = " ".join(line.rstrip() for line in myfile)

In [ ]:
print(data[:900])

In [ ]:
with open("resources/polish_stopwords.txt", encoding="utf8") as myfile:
    stop_text = " ".join(line.rstrip() for line in myfile)

stopWords = set(stop_text.split(" "))


In [ ]:
# text = re.sub(r"  [A-Z]  ", "Rozdział", data)
text = re.sub(r"[^\w]+", " ", data).split(" ")

chapters = [[]]

i = 0

for word in text:
    if word == "Rozdział":
        chapters.append([])
        i += 1
        
    else:
        if word not in stopWords and len(word) > 1 and word[0] != "'":
            chapters[i].append(word.lower())
    
print(chapters[1])
chapters = chapters[1:]
    

In [ ]:
count_all_chapters = {}
for word in [word for chapter in chapters for word in chapter]:
    count_all_chapters.setdefault(word, 0)
    count_all_chapters[word] = count_all_chapters[word] + 1

In [ ]:
chapters_tfs = []
chapters_word_counts = []
book_word_counts = {}

for chapter in chapters[1:]:
    chapter_count = {}
    total = len(chapter)
    
    for word in chapter:
        chapter_count.setdefault(word, 0)
        chapter_count[word] = chapter_count[word] + 1
        
        book_word_counts.setdefault(word, 0)
        book_word_counts[word] = book_word_counts[word] + 1
        
    chapters_word_counts.append(chapter_count)     
    chapters_tfs.append({word:(count/total) for (word, count) in chapter_count.items()})


In [ ]:
chapters_tf_idfs = []

for chapter_tf in chapters_tfs:
    chapter_tf_idf = {}
    for word, tf in chapter_tf.items():
        docs_with_term = np.sum([1 for chap in chapters_tfs if word in chap.keys()])
        chapter_tf_idf.setdefault(word, tf*np.log(len(chapters_tfs)/docs_with_term))
    
    chapters_tf_idfs.append(chapter_tf_idf)

In [ ]:
# print(sorted([x for x in chapters_tf_idfs[13].items()], key=lambda x: x[1], reverse=True))

In [ ]:
import os

def save_word_cloud(chapter, number_of_chapter=0, name=""): 
    
    if 'results' not in os.listdir():
        os.mkdir('results')
    
    # in windows change to \\
    path = os.getcwd() + '/results'
    counts_list = [(word, c) for word, c in zip(chapter.keys(), chapter.values())]
    counts_list.sort(key = lambda x: x[1], reverse=True)
    wc_dict = {w: c for w,c in counts_list[:64]}
    mask = np.array(Image.open("resources/storm.png"))
    cloud = WordCloud(background_color="white", mask=mask, max_words=100)
    cloud.generate_from_frequencies(wc_dict)
    plt.imshow(cloud, interpolation='bilinear')
    plt.axis("off")
    if name == "":
        cloud.to_file("{}/chapter_{}_cloud.png".format(path, number_of_chapter))
    else:
        cloud.to_file("{}/{}.png".format(path, name))  

In [ ]:
for chap_n, chapter in enumerate(chapters_word_counts):
    save_word_cloud(chapter, chap_n+1)

In [ ]:
save_word_cloud(book_word_counts, name='whole_book')

# Task 2

In [ ]:
chapters_highest_tf_idfs = []

for chapter in chapters_tf_idfs:
    chap_words = [item for item in chapter.items()]
    chapters_highest_tf_idfs.append(sorted(chap_words, key=lambda x: x[1], reverse=True)[:20])


In [ ]:
print(chapters_highest_tf_idfs[0])

In [ ]:
def find_chapters(word, chapters):
    chaps_list = []
    for i,chapter in enumerate(chapters):
        chaps_list.append((i, chapter.get(word.lower(), 0)))
    
    return sorted(chaps_list, key=lambda x: x[1], reverse=True)

In [ ]:
find_chapters("cezar", chapters_tf_idfs)